# 1. Generate Interferece Graph

# 2. Make Prediction from DL4RegAlloc

In [1]:
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib as plt
import pandas as pd
from keras.utils import to_categorical
seed(1)
tf.random.set_seed(2)

2023-11-26 00:30:48.267043: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-26 00:30:48.497325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 00:30:48.497442: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 00:30:48.533730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 00:30:48.611003: I tensorflow/core/platform/cpu_feature_guar

### Build Model

In [2]:
seqsize = 100
seq_inputs = layers.Input(shape=(seqsize,100,), dtype='float32')
encoder = layers.Bidirectional(layers.LSTM(512, return_sequences=True, name="lstm_1"))(seq_inputs)
encoder = layers.Dropout(0.2)(encoder)
encoder = layers.Bidirectional(layers.LSTM(256, return_sequences=True, name="lstm_2"))(encoder)
encoder = layers.Dropout(0.2)(encoder)
decoder = layers.Bidirectional(layers.LSTM(128, return_sequences=True, name="lstm_3"))(encoder)
decoder = layers.Dropout(0.2)(decoder)
decoderoutputs = layers.TimeDistributed(layers.Dense(101,activation="softmax"))(decoder)
model = tf.keras.Model(inputs=seq_inputs, outputs=decoderoutputs)

2023-11-26 00:30:49.909472: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-26 00:30:50.004779: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-26 00:30:50.005226: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### Load Checkpoint

In [3]:
model = tf.keras.models.load_model("DL4RegAlloc/saved_model/color3.h5")

### Utils & Post Process Function

In [4]:
# ============================== Utils =========================================

def plot_loss_accuracy(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['categorical_accuracy'])
    if ('val_categorical_accuracy' in history.history.keys()):
        plt.plot(history.history['val_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    if ('val_loss' in history.history.keys()):
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def adBits(x, x2, y):
   for i in range(x.shape[0]):
      #print(' for each train sample  ... ', i)
      loop = seqsize
      j = 0
      for loop in range(seqsize):     
          #print(' for each node ... ', j)
          adBits_1 = int(x[i][2*loop])
          #Get the next LONG since each node now has 2 LONGs for adj edges
          adBits_2 = int(x[i][2*loop+1])
          # Uncomment below if you want invalid nodes to not feed to NN before any valid nodes
          #if (adBits_1==0 and adBits_1==0 and loop!=seqsize-1):              
          #    y[i][j] = y[i][loop+1] 
          #    continue
          if (adBits_1):
              #print adBits
              for k in range(0,64):
                  bit = adBits_1 & 1
                  #print ( ' adBits&1 = ',  bit )
                  # Uncomment below if you want new nodes to only have adjacency to earlier nodes
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_1 >>= 1
              x2[i][j][j] = 1 

          if (adBits_2):   
              #print adBits
              for k in range(64,100):
                  bit = adBits_2 & 1
                  #print ( ' adBits&1 = ',  bit )
                  if ( bit == 1):
                  #if ( bit == 1 and k<j):
                      x2[i][j][k] = 1
                      #print x2[i-1][j-1][k-1]
                  else:
                      x2[i][j][k] = 0
                      #print x2[i-1][j-1][k-1]
                  adBits_2 >>= 1
              x2[i][j][j] = 1
          j = j+1
   return x2, y

def updateLabelBits(x, y):
    for i in range(x.shape[0]):
        #print(' for each train sample  ... ', i)
        label_dict = {}
        label_num = 1
        for j in range(seqsize):
            if (x[i][j][j] == 0):
                y[i][j] = 0
            else:
                if y[i][j] in label_dict:
                    y[i][j] = label_dict[y[i][j]]
                else:
                    label_dict[y[i][j]] = label_num
                    y[i][j] = label_num
                    label_num = label_num + 1
    return y

# ============================== Post Process ==================================

csv_rows = []

def post_process (x2_pred, predicted):
    #Calculate the number of edges which will require correction
    invCols = 0
    edges = 0
    for i in range(x2_pred.shape[0]):
        for j in range(seqsize): # row
            for k in range(j): # col
                adj = x2_pred[i][j][k]
                if (adj == 1):
                    edges += 1
                    if (np.argmax(predicted[i][j]) == np.argmax(predicted[i][k])):
                        invCols += 1
                        
    print('Total No of edges ', edges)
    print('# of edges with invalid coloring ', invCols)
    print('Total percentage of edges with invalid colors ', invCols/edges)

def post_process_chromatic (x2_pred, predicted):  
    colors_list_list = []
    for i in range(x2_pred.shape[0]):
        colors_list = []
        for j in range(seqsize):
            # Valid nodes will have below set to 1 so check the color 
            # assignment of those nodes only
            if (x2_pred[i][j][j] != 0):
                colors_list.append(np.argmax(predicted[i][j]))
        print('Colors list of graph ', i, ' is  \n', colors_list)
        chromatic_number = len(set(colors_list))
        print('Chromatic number of graph ', i, ' is  ', chromatic_number)
        colors_list_list.append(colors_list)
    return colors_list_list

def create_csv_rows (graph_name, colors_list_list_before_correction, 
                     colors_list_list_after_correction):    
    for i in range(len(colors_list_list_before_correction)):
        row = [graph_name, i, len(set(colors_list_list_before_correction[i])), 
               len(set(colors_list_list_after_correction[i]))]        
        csv_rows.append(row)

def post_process_correction (x2_pred, predicted, colors_list_list): 
  totInvCols = 0
  totEdges = 0

  for i in range(x2_pred.shape[0]):
      #maxcol = max(xpredicted[i])
      maxcol = max(colors_list_list[i])
      #print(maxcol)
      #mcol = maxcol[0]
      maxorigcol = maxcol
      mcolnew = maxcol
      #print('Maxcol = ',maxcol[0])
      #print(' ... FOR SAMPLE  ... ', i)
      invCols = 0
      edges = 0;
      newCol = 500

      for j in range(seqsize):
          #print(' ... ... FOR EACH NODE ... ...', j)
          for k in range(j):
              #print(' ... ... ... for each adjacency  ... ... ...', k)
              adj = x2_pred[i][j][k]
              #There is an edge
              if ( adj == 1 ):
                  edges += 1
                  if ( np.argmax(predicted[i][j]) == np.argmax(predicted[i][k]) ):                   
                      col_j = np.argmax(predicted[i][j])
                      col_k = np.argmax(predicted[i][k])
                      invCols += 1

                      #Check whether we can give one of the existing colors
                      foundfinalcol = 0
                      for  y in range(1,maxcol+1):
                          #print('Check for COLOR NO ... ', y)
                          if ( foundfinalcol == 1 ) :
                              #print('FOUND COLOR ALREADY  ... leave the loop')
                              break

                          foundcol = 0
                          #Check the adjacent nodes of j
                          #for  z in range(j):
                          for z in range(seqsize):
                              if j!=z:
                                  if  (   ((x2_pred[i][j][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                      or  ((x2_pred[i][z][j] == 1) and (np.argmax(predicted[i][z]) == y))
                                      ):
                                      #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                      foundcol = 1
                                      #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                      break

                          #Finished checking the adjacent nodes of j
                          #Color y is not used by any of j's neighbours
                          #print('[1] Finished Checking the adjacent node of ... ',j,' ... foundcol = ',foundcol)
                          if ( foundcol == 0 ) :
                              #assign any prediction > 1
                              predicted[i][j][y] = 2
                              #print('[1] Reuse color ', y, ' for node ', j)
                              foundfinalcol = 1

                          else :
                              foundcol = 0                                                            
                              #Check the adjacent nodes of k
                              for z in range(seqsize):
                                  if k!=z:
                                      if  (   ((x2_pred[i][k][z] == 1) and (np.argmax(predicted[i][z]) == y))
                                          or  ((x2_pred[i][z][k] == 1) and (np.argmax(predicted[i][z]) == y))
                                          ):
                                          #print('[1] Adjacent node ... from ',j, '-->', z, 'color = ',xpredicted[i][z][0] )
                                          foundcol = 1
                                          #print('[1] Found Color ', y, ' for node ', z, 'from node ', j )
                                          break
                              #Color y is not used by any of k's neighbours
                              if ( foundcol == 0 ) :
                                  #assign any prediction > 1
                                  predicted[i][k][y] = 2
                                  #print('[2] Reuse color ', y, ' for node ', k )
                                  foundfinalcol = 1

                      # Could not color using an existing color
                      # Get a new color from 500 onwards OR use from the new 500 color number series
                      if ( foundfinalcol == 0 ) :
                           #newCol += 1
                           mcolnew += 1
                           #assign any prediction > 1
                           predicted[i][k][mcolnew] = 2
                           maxcol +=1
                           #print('Use new color ', mcolnew, ' for node ', k)

  #print('No of edges with invalid coloring while color correction is ON ', invCols)
  #totInvCols = totInvCols + invCols
  #print('No of edges ', edges)
  #totEdges = totEdges + edges
  #print('percentage of edges with invalid colors found by color correction algo', invCols/edges)

  #print('Ground truth cols = ',tncols[i][0])
  #print('Total colors predicted before compensation', maxorigcol )
  #print('Total colors predicted after compensation', mcolnew)
  #print('Extra colors', mcolnew - tncols[i][0])
  return predicted



### Process Input

In [12]:
test_csvs =["DL4RegAlloc/graph_csvs/baidu.csv", "DL4RegAlloc/graph_csvs/gkarate.csv"]

x_pred_list = []
y_pred_list = []

for i,csv in enumerate(test_csvs):
    print(csv, "information:")
    tseq = pd.read_csv(csv,header=None)
    
    tstcolumns = list(tseq)
    
    tedataadj = tseq[tstcolumns[1:2*seqsize+1]]
    tedatacols = tseq[tstcolumns[2*seqsize+1:3*seqsize+1]]
    
    x_pred = np.array(tedataadj)
    y_pred = np.array(tedatacols)
    x2_pred = np.zeros((x_pred.shape[0],y_pred.shape[1],seqsize))
    x2_pred, y_pred = adBits(x_pred, x2_pred, y_pred)
    y_pred  = updateLabelBits(x2_pred, y_pred)
    y2_pred = to_categorical(y_pred, num_classes=101)

    x_pred_list.append(list(x2_pred))
    y_pred_list.append(list(y2_pred))

    print(np.asarray(x_pred_list[i]).shape)
    print(np.asarray(y_pred_list[i]).shape)  

DL4RegAlloc/graph_csvs/baidu.csv information:
(1, 100, 100)
(1, 100, 101)
DL4RegAlloc/graph_csvs/gkarate.csv information:
(1, 100, 100)
(1, 100, 101)


### Prediction

In [19]:
# TESTING
for i,csv in enumerate(test_csvs):
    print('\n------PREDICTING ',csv,'-------')
    predicted = model.predict(np.asarray(x_pred_list[i]), batch_size=None, 
                              verbose=0, steps=None, callbacks=None)
    print(predicted.shape)
            
    print('\nInvalid edges percentage before color correction ->')
    post_process(np.asarray(x_pred_list[i]), predicted)
    
    print('\nColors list and Chromatic number predicted by the model ->')
    colors_list_list_before_correction = post_process_chromatic(np.asarray(x_pred_list[i]), predicted)
    print('\nApply color correction ->')
    predicted = post_process_correction(np.asarray(x_pred_list[i]), predicted, colors_list_list_before_correction)
    print('\nColors list and Chromatic number following color correction ->')
    colors_list_list_after_correction = post_process_chromatic(np.asarray(x_pred_list[i]), predicted)
    print('\nInvalid edges percentage after color correction ->')
    post_process(np.asarray(x_pred_list[i]), predicted)
    create_csv_rows(csv.rsplit('/',1)[1], colors_list_list_before_correction, colors_list_list_after_correction)

import csv
with open("Result/graph_coloring_result.csv", 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)      
    # writing the data rows
    csvwriter.writerows(csv_rows)


------PREDICTING  DL4RegAlloc/graph_csvs/baidu.csv -------
(1, 100, 101)

Invalid edges percentage before color correction ->
Total No of edges  91
# of edges with invalid coloring  31
Total percentage of edges with invalid colors  0.34065934065934067

Colors list and Chromatic number predicted by the model ->
Colors list of graph  0  is  
 [2, 3, 2, 3, 1, 4, 4, 1, 3, 3, 3, 1, 2, 3, 3, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 4, 3, 3, 1, 1, 1, 3, 2, 1, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 1, 2, 1, 3, 2, 3]
Chromatic number of graph  0  is   4

Apply color correction ->

Colors list and Chromatic number following color correction ->
Colors list of graph  0  is  
 [1, 2, 2, 1, 1, 4, 4, 1, 3, 3, 3, 1, 3, 3, 3, 2, 2, 2, 1, 2, 3, 2, 1, 3, 2, 1, 2, 2, 1, 1, 3, 3, 1, 1, 4, 3, 1, 2, 1, 1, 1, 2, 1, 3, 2, 3, 1, 3, 2, 3, 2, 1, 2, 3, 1, 2, 1, 3, 2, 3]
Chromatic number of graph  0  is   4

Invalid edges percentage after color correction ->
Total No of edges  91
# of edges with invalid 

# 3. RegAlloc in LLVM